In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline
%reload_ext autoreload
%autoreload

In [9]:
# loading in robot dataset
import tsfresh.examples.robot_execution_failures as robot
robot.download_robot_execution_failures()

In [11]:
import os
robot.data_file_name # provides functions to work with directories

'C:\\Users\\Bane6\\anaconda3\\envs\\dslab\\lib\\site-packages\\tsfresh\\examples\\data\\robotfailure-mld\\lp1.data'

In [13]:
ts, label = robot.load_robot_execution_failures() # separating into input signals and associated labels

In [27]:
robot.load_robot_execution_failures()

(      id  time  F_x  F_y  F_z  T_x  T_y  T_z
 0      1     0   -1   -1   63   -3   -1    0
 1      1     1    0    0   62   -3   -1    0
 2      1     2   -1   -1   61   -3    0    0
 3      1     3   -1   -1   63   -2   -1    0
 4      1     4   -1   -1   63   -3   -1    0
 ...   ..   ...  ...  ...  ...  ...  ...  ...
 1315  88    10  -10    2   39  -21  -24    5
 1316  88    11  -11    2   38  -24  -22    6
 1317  88    12  -12    3   23  -24  -24    5
 1318  88    13  -13    4   26  -29  -27    5
 1319  88    14  -13    2   15  -25  -25    6
 
 [1320 rows x 8 columns],
 1      True
 2      True
 3      True
 4      True
 5      True
       ...  
 84    False
 85    False
 86    False
 87    False
 88    False
 Length: 88, dtype: bool)

In [14]:
ts # input time series signals

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0
...,...,...,...,...,...,...,...,...
1315,88,10,-10,2,39,-21,-24,5
1316,88,11,-11,2,38,-24,-22,6
1317,88,12,-12,3,23,-24,-24,5
1318,88,13,-13,4,26,-29,-27,5


In [15]:
label # associated input labels

1      True
2      True
3      True
4      True
5      True
      ...  
84    False
85    False
86    False
87    False
88    False
Length: 88, dtype: bool

In [23]:
from tsfresh import extract_features

X = extract_features(ts, column_id='id', column_sort='time')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:15<00:00,  1.29it/s]


In [29]:
X.isna().sum() # find missing data, if any present, remove them

F_x__variance_larger_than_standard_deviation              0
F_x__has_duplicate_max                                    0
F_x__has_duplicate_min                                    0
F_x__has_duplicate                                        0
F_x__sum_values                                           0
                                                         ..
T_z__matrix_profile__feature_"mean"__threshold_0.98      88
T_z__matrix_profile__feature_"median"__threshold_0.98    88
T_z__matrix_profile__feature_"25"__threshold_0.98        88
T_z__matrix_profile__feature_"75"__threshold_0.98        88
T_z__mean_n_absolute_max__number_of_maxima_7              0
Length: 4734, dtype: int64

In [35]:
Xreal = X.dropna(axis=1) # by default, .dropna() drops all rows (need to define `axis=1` to remove cols)
Xreal.shape

(88, 2215)

In [36]:
from tsfresh.transformers import FeatureSelector # an obj which has implemented the API of circuit learn

select = FeatureSelector() # learning function

In [38]:
select.fit(Xreal, label) # the fn has learnt about the statistically significant features for distinguishing these labels

FeatureSelector()

In [40]:
p_values = pd.Series(select.p_values, index=select.features) # have a look at the statistically sig results
p_values

F_x__value_count__value_-1                                         1.879633e-12
F_x__root_mean_square                                              5.900086e-12
F_x__abs_energy                                                    5.900086e-12
T_y__absolute_maximum                                              5.966395e-12
F_x__mean_n_absolute_max__number_of_maxima_7                       6.054628e-12
                                                                       ...     
T_z__agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"var"             NaN
T_z__ratio_beyond_r_sigma__r_5                                              NaN
T_z__ratio_beyond_r_sigma__r_6                                              NaN
T_z__ratio_beyond_r_sigma__r_7                                              NaN
T_z__ratio_beyond_r_sigma__r_10                                             NaN
Length: 2215, dtype: float64

Look for the two underscores:
- everything before the first set of two underscores is the "name of the Time Series" used for calculating the feature
- everything after the second set of two underscores is the "function name"
- everything after the third set of two underscores is the "parameters"

##  Find a really good feature for discriminating the labels

In [46]:
Xreal[select.relevant_features].shape

(88, 675)

In [45]:
select.transform(Xreal).shape

(88, 675)

In [50]:
np.alltrue(Xreal[select.relevant_features] == select.transform(Xreal))

True

In [43]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(Xreal, )

TypeError: fit() missing 1 required positional argument: 'y'